In [1]:
import pandas as pd
from google.colab import files


train = pd.read_csv("/content/drive/MyDrive/Анализ Данных 2021-2022/4ое занятие/train_clever.csv")
test = pd.read_csv("/content/drive/MyDrive/Анализ Данных 2021-2022/4ое занятие/test_clever.csv")
test

,ID,Question
0,11083,В каком году основан Санкт-Петербург?
1,18349,Какой город Золотого кольца России-город худож...
2,11374,Каким цветом написана надпись магазина «пятеро...
3,9699,Сколько весит пирамида Хеопса?
4,17748,В какой стране киберспорт впервые признали вид...
...,...,...
9995,28782,В каком году началась Первая Мировая война?
9996,13015,Самое популярное женское имя
9997,26647,Сколько на данный момент снято официальных фил...
9998,9578,У какого айфона появился впервые появился Touc...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
print(train.shape)
print(test.shape)

(20000, 3)
(10000, 2)


In [4]:
train.head()

,ID,Answer,Question
0,16019,0,Крупнейшим производителем кукурузы является:
1,5121,0,Кто первый в РОССИЮ завёз картошку ?
2,17169,0,Кто из мореплавателей сделал первое кругосветн...
3,15480,0,Как зовут героя серии романов Бариса Акунина?
4,3312,0,Какая сборная выйграла чемпионат мира по футбо...


In [7]:
import nltk
import spacy
import re

In [9]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
def standardize_text(df, text_field ):
    df[text_field] = df[text_field].str.replace(r".", "")
    df[text_field] = df[text_field].str.replace(r"!", "")
    df[text_field] = df[text_field].str.replace(r"?", "")
    df[text_field] = df[text_field].str.replace(r":", "")
    df[text_field] = df[text_field].str.lower()
    return df

train = standardize_text(train, "Question")
test = standardize_text(test, "Question")
#train.to_csv("clean_data.csv")
train.head()


,Question
0,в каком году основан санкт-петербург
1,какой город золотого кольца россии-город худож...
2,каким цветом написана надпись магазина «пятеро...
3,сколько весит пирамида хеопса
4,в какой стране киберспорт впервые признали вид...


In [34]:
train=train.drop(["ID"], axis=1)
test=test.drop(["ID"], axis=1)

In [29]:
train.groupby("Answer").count()

,ID,Question
Answer,,
0,17885,17885
1,2115,2115


In [30]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

train["tokens"] = train["Question"].apply(tokenizer.tokenize)
train.head()

,ID,Answer,Question,tokens
0,16019,0,крупнейшим производителем кукурузы является,"[крупнейшим, производителем, кукурузы, является]"
1,5121,0,кто первый в россию завёз картошку,"[кто, первый, в, россию, завёз, картошку]"
2,17169,0,кто из мореплавателей сделал первое кругосветн...,"[кто, из, мореплавателей, сделал, первое, круг..."
3,15480,0,как зовут героя серии романов бариса акунина,"[как, зовут, героя, серии, романов, бариса, ак..."
4,3312,0,какая сборная выйграла чемпионат мира по футбо...,"[какая, сборная, выйграла, чемпионат, мира, по..."


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vectorizer = CountVectorizer()


list_corpus = train["Question"].tolist()
list_labels = train["Answer"].tolist()
test_corpus = test["Question"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts = count_vectorizer.transform(test_corpus)

In [33]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

list_corpus = train["Question"].tolist()
list_labels = train["Answer"].tolist()
test_corpus = test["Question"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts = count_vectorizer.transform(test_corpus)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [37]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')

    roc_auc = roc_auc_score(y_test, y_predicted_counts)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)

    return accuracy, precision, recall, f1, roc_auc

accuracy, precision, recall, f1, roc_auc = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f, roc_auc = %.3f" % (accuracy, precision, recall, f1, roc_auc))

accuracy = 0.798, precision = 0.826, recall = 0.798, f1 = 0.811, roc_auc = 0.533


In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')

    roc_auc = roc_auc_score(y_test, y_predicted_counts)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)

    return accuracy, precision, recall, f1, roc_auc

accuracy, precision, recall, f1, roc_auc = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f, roc_auc = %.3f" % (accuracy, precision, recall, f1, roc_auc))

accuracy = 0.798, precision = 0.826, recall = 0.798, f1 = 0.811, roc_auc = 0.533


In [40]:
y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)

y_test_pred

,index,Answer
0,0,0
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,0
9997,9997,0
9998,9998,0


In [42]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [45]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [48]:
y_predicted_test = clf_tfidf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)

y_test_pred

,index,Answer
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,0
9997,9997,0
9998,9998,1


In [ ]:
y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)

y_test_pred

In [49]:
y_test_pred.to_csv('submition_test_Yana_draft1.csv', index=False)
files.download("submition_test_Yana_draft1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>